# Kubeflow Setup

- It's time to get started with Kubeflow!
- We will see how to install Kubeflow

### Installing Kubeflow

- Kubeflow works on Kubernetes, and therefore, we can develop our application locally, and move on to deploying it to a Kubernetes cluster eventually.
    - For example, you can run everything on minikube, and then deploy the same application to EKS. However, not all computers have the capacity for running a large cluster locally, so you might need to directly deploy to EKS.
    - We need to first install kubectl to use Kubernetes locally.
        - You can review the Kubernetes content to see how to install kubectl.

In this notebook, we will guide you through the process of creating a cluster on EKS, and how to install Kubeflow on it.

> <font size=+1> Attention! The following guide will create a new EKS cluster, which doesn't use EC2 instances that belong to the free tier. </font>

If you don't want to spend money, we recommend you to use the provided AWS credentials and follow the instructions given in this file for the [Facebook Marketplace's scenario](https://aicore-files.s3.amazonaws.com/MLOps/Facebook_Setup.md). Run the commands included **in the EC2 instance**, not locally

The following steps are a guide to understand how Kubeflow is setup on EKS, so in case you want or need to understand the process, we recommend you to follow the steps below.

## Creating a cluster on EKS

First of all, what is EKS?

> <font size=+1> EKS (Elastic Kubernetes Service) is an AWS service that allows orchestrating Docker containers on AWS</font>

Since Kubeflow runs on Kubernetes, EKS is a great tool to run your application on AWS.

<p align=center><img src=images/EKS_1.png width=700></p>

You can create your cluster using the AWS Management Console, but we encourage you to use the command line interface, since eventually you will need it to install Kubeflow on the created cluster.

Before creating the EKS cluster, you need to install the following tools that you will use on the command line:
- __eksctl__: This tool allows you to create and manage Kubernetes clusters on EKS.
- __kubectl__: Your cluster will have an API server, and you will need to use this tool to interact with it.
- __AWS CLI__: The AWS CLI tool is used to interact with AWS services.

Additionally, you have to create and IAM user with the following permissions:
- EKS roles
- CloudFormation, since the EKS cluster is actually created from a stack
- VPC related resources

Take a look at this [link](https://docs.aws.amazon.com/service-authorization/latest/reference/list_amazonelastickubernetesservice.html) to know more about Kubernetes permissions.

If only you are going to use this service, you can simply create an IAM role with all permissions instead.

### Installing _eksctl_

<details>
  <summary>For Ubuntu Users</summary>

  1. Download and extract eksctl using the following commands:
  ```
      curl --silent --location "https://github.com/weaveworks/eksctl/releases/latest/download/eksctl_$(uname -s)_amd64.tar.gz" | tar xz -C /tmp

      sudo mv /tmp/eksctl /usr/local/bin
  ```

  2. Verify that eksctl is installed correctly:
  ```
      eksctl version
  ```
</details>

<details>
  <summary>For Mac Users</summary>

  1. Install Homebrew. If you haven't follow the following [instructions](https://brew.sh/)
  2. Install the Weaveworks Homebrew tap running the following command:
  ```
    brew tap weaveworks/tap
  ```
  3. Install eksctl with the following command
  ```
    brew install weaveworks/tap/eksctl
  ```
  4. Check that eksctl was installed correctly:
  ```
    eksctl version
  ```

</details>

<details>
  <summary>For Windows Users</summary>

  1. Install Chocolatey. If you haven't follow the following [instructions](https://chocolatey.org/install)
  2. Install the binaries with the following command:
  ```
    choco install -y eksctl
  ```
  3. Check that eksctl was installed correctly:
  ```
    eksctl version
  ```
</details>

When you finish, you should see something like this:

<p align=center><img src=images/EKSCTL_1.png width=400></p>

### Installing _kubectl_

<details>
  <summary>For Ubuntu Users</summary>

  1. Download the kubectl binary from AWS. Here, we are downloading version 1.21 because it was the stable one when writing this notebook. But you can check different versions [here](https://docs.aws.amazon.com/eks/latest/userguide/install-kubectl.html). 
  ```
      curl -o kubectl https://s3.us-west-2.amazonaws.com/amazon-eks/1.21.2/2021-07-05/bin/linux/amd64/kubectl
  ```

  2. Change the permissions to the binary file:
  ```
      chmod +x ./kubectl
  ```
  3. Copy the binary to a folder in your PATH
  ```
      mkdir -p $HOME/bin && cp ./kubectl $HOME/bin/kubectl && export PATH=$PATH:$HOME/bin
  ```
  4. You can add the binary to the shell initialization:
  ```
      echo 'export PATH=$PATH:$HOME/bin' >> ~/.bashrc
  ```
  5. Verify that kubectl is installed correctly:
  ```
      kubectl version --client
  ```
</details>

<details>
  <summary>For Mac Users</summary>

  1. Download the kubectl binary from AWS. Here, we are downloading version 1.21 because it was the stable one when writing this notebook. But you can check different versions [here](https://docs.aws.amazon.com/eks/latest/userguide/install-kubectl.html). 
  ```
      curl -o kubectl https://s3.us-west-2.amazonaws.com/amazon-eks/1.21.2/2021-07-05/bin/darwin/amd64/kubectl
  ```

  2. Change the permissions to the binary file:
  ```
      chmod +x ./kubectl
  ```
  3. Copy the binary to a folder in your PATH
  ```
      mkdir -p $HOME/bin && cp ./kubectl $HOME/bin/kubectl && export PATH=$HOME/bin:$PATH
  ```
  4. You can add the binary to the shell initialization:
  ```
      echo 'export PATH=$PATH:$HOME/bin' >> ~/.bash_profile
  ```
  5. Verify that kubectl is installed correctly:
  ```
      kubectl version --client
  ```

</details>

<details>
  <summary>For Windows Users</summary>

  1. Download the kubectl binary from AWS. Here, we are downloading version 1.21 because it was the stable one when writing this notebook. But you can check different versions [here](https://docs.aws.amazon.com/eks/latest/userguide/install-kubectl.html). 
  ```
      curl -o kubectl.exe https://s3.us-west-2.amazonaws.com/amazon-eks/1.21.2/2021-07-05/bin/windows/amd64/kubectl.exe
  ```
  2. Copy the binary to a folder in your PATH
  
  $\qquad$ a. Create a new directory for your command line binaries, such as `C:\bin`

  $\qquad$ b. Copy the binary to the new directory

  $\qquad$ c. Edit your user or system PATH environment variable to add the new directory to your PATH
  
  $\qquad$ d. Restart your terminal
  
  3. Verify that kubectl is installed correctly:
  ```
      kubectl version --client
  ```

</details>

When you finish, you should see something like this:

<p align=center><img src=images/KUBECTL_1.png width=500></p>

### Installing _aws cli_

As long as you have installed pip in your computer, you can install AWS CLI using the following command:

```
    pip install awscli
```


### Creating the IAM user

You need to create the IAM user that will enable the EKS cluster to communicate with the AWS services. If you already know how to create and IAM user, and authenticate with the AWS CLI, you can skip this section.

First, in your AWS console, go to the IAM console and create a new user.

<p align=center>
  <img src=images/IAM_1.png width=150>
</p>

Click on "Add users"

<p align=center>
    <img src=images/IAM_2.png width=500>
</p>

Give any name to your user, and select "Access Key - Programmatic access" for "Select AWS credential type", and then click "Next"

<p align=center>
    <img src=images/IAM_3.png width=500>
</p>

Then, , click on "Attach existing policies directly" and select "AdministratorAccess". Click "Next" until you create the user

<p align=center>
    <img src=images/IAM_4.png width=500>
</p>

Download the `csv` file with your Access key ID and Secret access key. Once you close this window, you won't be able to check the secret access key again from the page, so make sure you downloaded the file before closing the window.

#### Authenticating with the AWS CLI

On your terminal, type `aws configure`. This will ask you for your Access key ID, Secret access key, and a region. Don't forget the region you are using now, you will have to use the same region when creating the cluster.

<p align=center>
    <img src=images/IAM_5.png width=500>
</p>

### Creating the EKS cluster

Now that everything is installed, you can create the cluster that will be used in this notebook.

The syntax for creating a cluster is as follows:

- `eksctl create cluster [flags]`

The main flags you need to use are:

- `--name`: The name of the cluster
- `--region`: The region where the cluster will be created. Use the same region you are using to create the IAM user.
- `--nodegroup-name`: The name of the node group
- `--node-type`: The type of node that will be created. For installing Kubeflow, we will, use `t2.xlarge`
- `--nodes`: The number of nodes that will be created. For installing Kubeflow, we will, use 2
- `--timeout`: maximum waiting time for any long-running operation to complete. In this case, we will use `40m` (40 minutes)
- `--version`: The Kubernetes version that will be used. For installing Kubeflow, we will, use `1.19` because we know it's stable, and recent versions have been causing issues with Kubeflow

You can always check these flags by running `eksctl create cluster --help`

The whole command will look like:



```
eksctl create cluster \
    --name kubeflow-cluster \
    --region us-east-2 \
    --nodegroup-name kubeflow-node-group \
    --node-type t2.xlarge \
    --nodes 2 \
    --timeout 40m \
    --version 1.19
```

Creating the cluster takes a bit of time, so you can start working on other things while the cluster is being created, like installing kustomize (see next section). It might take around 20-30 minutes, and the command line will look like this:

<p align=center><img src=images/EKS_2.png width=600></p>

One thing to take into account is that, when you create a cluster, eksctl will modify your kubeconfig file, so next time you apply a manifest in your local machine, you will actually apply that manifest to the cluster you are now creating. 

You can always change where to apply the manifest changing the `--kubeconfig` flag, or modifying the `~/.kube/config` file.

## Installing Kubeflow in the cluster

Before installing Kubeflow, you need to install `kustomize`, which is a command line tool that allows you to modify the manifest files of Kubernetes. Depending on you local machine, in order to use `kustomize`, you need to install `yq` and `jq` which are command line tools for processing YAML and JSON files respectively. It doesn't hurt to install them, so just in case, let's see how to install them.



### Installing _yq_



<details>
  <summary>For Ubuntu Users</summary>

  1. Download the latest executable file of yq from GitHub
  ```
      sudo wget -qO /usr/local/bin/yq https://github.com/mikefarah/yq/releases/latest/download/yq_linux_amd64
  ```

  2. Change the permissions to the binary file:
  ```
      chmod a+x /usr/local/bin/yq
  ```
  3. Verify that yq is installed correctly:
  ```
      yq --version
  ```
</details>

<details>
  <summary>For Mac Users</summary>

  1. You can simply use Homebrew to install it
  ```
      brew install yq
  ```
  2. Verify that yq is installed correctly:
  ```
      yq --version
  ```
</details>

<details>
  <summary>For Windows Users</summary>

  1. You can simply use Chocolatey to install it
  ```
      choco install yq
  ```
  2. Verify that yq is installed correctly:
  ```
      yq --version
  ```
</details>

### Installing _jq_


<details>
  <summary>For Ubuntu Users</summary>

  1. Update the package lists and install jq
  ```
      sudo apt update
      sudo apt install -y jq
  ```
  2. Verify that jq is installed correctly:
  ```
      jq --version
  ```
</details>

<details>
  <summary>For Mac Users</summary>

  1. You can simply use Homebrew to install it
  ```
      brew install jq
  ```
  2. Verify that jq is installed correctly:
  ```
      jq --version
  ```
</details>

<details>
  <summary>For Windows Users</summary>

  1. You can simply use Chocolatey to install it
  ```
      choco install jq
  ```
  2. Verify that jq is installed correctly:
  ```
      jq --version
  ```
</details>

### Installing _kustomize_

> <font size=+1> Attention! The latest version of kustomize doesn't work with latest versions of Kubeflow. You will need to use kustomize 3.2.0 </font>

<details>
  <summary>For Ubuntu Users</summary>

  1. Download the corresponding version and binary:
  ```
      wget -O kustomize https://github.com/kubernetes-sigs/kustomize/releases/download/v3.2.0/kustomize_3.2.0_linux_amd64
  ```
  2. Change the permissions
  ```
      chmod +x kustomize
  ```
  3. Move the binary to PATH
  ```
      sudo mv kustomize /usr/local/bin/
  ```
  4. Verify that kustomize is installed correctly:
  ```
      kustomize version
  ```
</details>

<details>
  <summary>For Mac Users</summary>

  1. Download the corresponding version and binary:
  ```
      wget -O kustomize https://github.com/kubernetes-sigs/kustomize/releases/download/v3.2.0/kustomize_3.2.0_darwin_amd64
  ```
  2. Change the permissions
  ```
      chmod +x kustomize
  ```
  3. Move the binary to PATH
  ```
      sudo mv kustomize /usr/local/bin/
  ```
  4. Verify that kustomize is installed correctly:
  ```
      kustomize version
  ```
</details>

<details>
  <summary>For Windows Users</summary>

  Windows users could install kustomize from Chocolatey. But unfortunately, chocolatey only downloads the latest version of kustomize.

  You can follow this StackOverflow thread to get kustomize 3.2.0 installed. You will need to install GO...
  https://stackoverflow.com/questions/70838480/build-kustomize-3-2-0-on-windows
</details>

### Installing Kubeflow

As mentioned, Kubeflow works using Kubernetes manifests. You can download the latest version of these manifests from the [Kubeflow GitHub repository](https://github.com/kubeflow/manifests), and then checking out to the latest version

> The version of the manifest as of this writing is `v1.5`. You can check the latest version on the repository
```
git clone https://github.com/kubeflow/manifests
cd manifests
git checkout v1.5-branch
```

Now, we will use `kustomize` to build all the Kubeflow components and services:

```
while ! kustomize build example | kubectl apply -f -; do echo "Retrying to apply resources"; sleep 10; done
```

The `while` loop will keep trying to apply the resources until it succeeds. This is because of how Kubernetes and `kubectl` work, since they may fail on the first try.

This will also take around 20-30 minutes, don't worry about all the warnings and retries you will see.

<p align=center><img src=images/KUBECTL_2.png width=600></p>

The process will run until everything is configured. Once again, this is because of how Kubernetes and `kubectl` work, since they may fail on the first try.

## Connect to Kubeflow

After all this long process, you should be able to connect to Kubeflow. Run the following command to expose the Kubeflow dashboard:

```
kubectl port-forward svc/istio-ingressgateway -n istio-system 8080:80
```

The command line will show the port that you can access the dashboard on:

<p align=center><img src=images/KUBECTL_3.png width=600></p>

So, you can go to `http://localhost:8080` to access the dashboard.

When you installed Kubeflow, you used the default email and password given by the manifests you downloaded. There are:
- user: user@example.com
- password: 12341234

If you intend to deploy your application into production, you should change the password to something more secure. Take a look at this [section](https://github.com/kubeflow/manifests/tree/v1.5-branch#change-default-user-password) to know how to do so

If you see the following window, congratulations! You have successfully installed Kubeflow.

<p align=center><img src=images/Kubeflow.png width=600></p>

However, there is a small thing you still need to configure in order to use notebooks in Kubeflow. The problem is that Kubeflow is running currently on an HTTP protocol. This would not be necessary if you can run your notebooks on HTTPS. You can take a look at this [link](https://github.com/awslabs/kubeflow-manifests/issues/67#issuecomment-1059566247) to see how to do so

Here, we are going to show you how to run notebooks on HTTP.

First, stop exposing your EKS by pressing `ctrl + c`

Now, we need to change the `jupyter-web-app-deployment.yaml` file to expose the notebook on HTTP.

```kubectl edit deployments -n kubeflow jupyter-web-app-deployment```

This will open a VIM editor. Be careful now, a small change will break the Kubeflow configuration. Locate the `spec.template.spec.containers.env` section. It should look like this:

<p align=center><img src=images/KUBECTL_4.png width=400></p>

There, add the following lines. Remember that you are on VIM, you need to type "i" to insert new lines. Also, you are modifying a YAML file, sometimes, pressing TAB will not work, so use the spacebar to add those whitespaces:

```
  - name: APP_SECURE_COOKIES
    value: "false"
```

So, it should look like this now:

<p align=center><img src=images/KUBECTL_5.png width=400></p>

Save and exit (press "Esc" and type ":wq")

Restart the deployment for the changes to take effect

```kubectl -n kubeflow rollout restart deployment/jupyter-web-app-deployment```

And once again, expose the Kubeflow dashboard:

```
kubectl port-forward svc/istio-ingressgateway -n istio-system 8080:80
```

and go to `http://localhost:8080` to access the notebook.

Then, click on "Notebooks", and "New Notebook" to create a new notebook.

Give any name to it, and scroll down to the bottom of the page. Click on "Launch" to start the notebook. And this is what you should see:

<p align=center><img src=images/Kubeflow_2.png width=700></p>

If you can connect to it, you are done setting up Kubeflow!